In [11]:
# 获取格子状态

def get_state(row,col):
    if row != 3:
        return 'ground'
    if row == 3 and col == 0:
        return 'ground'
    if row == 3 and col == 11:
        return 'terminal'
    return 'trap'


In [12]:
def move(row,col,action):
	# 如果已经在悬崖，或者重点，反馈就是0
	if get_state(row,col) in ['trap','terminal']:
		return row,col,0
	# 👆
	if action == 0:
		row -= 1
	# 👇
	if action == 1:
		row += 1
	# 👈
	if action == 2:
		col -= 1
	# 👉
	if action == 3:
		col += 1
	#避免走到地图外面
	row = max(0,row)
	row = min(3,row)
	col = max(0,col)
	col = min(11,col)
	#如果是悬崖，奖励就是-100，否则都是-1
	reward = -1
	if get_state(row,col) == 'trap':
		reward = -100
	return row,col,reward

In [13]:
import numpy as np
#初始化Value和pi
values = np.zeros([4,12])
pi = np.ones([4,12,4])*0.25

In [14]:
# 动作价值函数
# 计算一个状态下执行动作的分数
def get_qsa(row,col,action):
    # 计算下一个状态的价值
    next_row , next_col,reward = move(row,col,action)
    # 计算下一个状态的分数，0.9是折扣因子
    value = values[next_row,next_col] * 0.9
    # 如果是悬崖或者陷阱，下一个状态分数是0
    if get_state(next_row,next_col) in ['trap','terminal']:
        value = 0
    return value + reward


## 策略评估

评估每一个格子的价值

In [15]:
def get_values():

    # 初始化一个新的values,重新评估所有格子的分数

    new_values = np.zeros([4,12])

   # 遍历所有格子 
    for row in range(4):
        for col in range(12):
            
            # 计算当前格子4个动作分别的分数
            action_value = np.zeros(4)

            # 遍历动作
            for action in range(4):
                action_value[action] = get_qsa(row, col, action)
            
            # 计算当前格子的分数
            action_value *= pi[row, col]

            # 所有动作分数求和
            new_values[row, col] = action_value.sum()

    return new_values

In [16]:
get_values()

array([[ -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,
         -1.  ,  -1.  ,  -1.  ,  -1.  ],
       [ -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,  -1.  ,
         -1.  ,  -1.  ,  -1.  ,  -1.  ],
       [ -1.  , -25.75, -25.75, -25.75, -25.75, -25.75, -25.75, -25.75,
        -25.75, -25.75, -25.75,  -1.  ],
       [-25.75,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,   0.  ,
          0.  ,   0.  ,   0.  ,   0.  ]])

## 策略提升

根据每个格子的价值重新计算每个动作的概率

In [17]:
def get_pi():
    # 重新初始化每个格子下采用动作的概率，重新评估
    new_pi = np.zeros([4,12,4])
    # 遍历
    for row in range(4):
        for col in range(12):
            
            action_value = np.zeros(4)

            for action in range(4):
                action_value[action] = get_qsa(row, col, action)
            
            # 计算每个动作的概率，根据达到最大值均分
            count = (action_value == action_value.max()).sum()

            for action in range(4):
                if action_value[action] == action_value.max():
                    new_pi[row,col,action] = 1/count
                else:
                    new_pi[row,col,action] = 0

    return new_pi

In [19]:
for i in range(10):
    for j in range(10):
        values = get_values()
    pi = get_pi()

In [20]:
values,pi

(array([[-7.71232075, -7.45813417, -7.17570464, -6.86189404, -6.5132156 ,
         -6.12579511, -5.6953279 , -5.217031  , -4.68559   , -4.0951    ,
         -3.439     , -2.71      ],
        [-7.45813417, -7.17570464, -6.86189404, -6.5132156 , -6.12579511,
         -5.6953279 , -5.217031  , -4.68559   , -4.0951    , -3.439     ,
         -2.71      , -1.9       ],
        [-7.17570464, -6.86189404, -6.5132156 , -6.12579511, -5.6953279 ,
         -5.217031  , -4.68559   , -4.0951    , -3.439     , -2.71      ,
         -1.9       , -1.        ],
        [-7.45813417,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ]]),
 array([[[0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0.5 , 0.  , 0.5 ],
         [0.  , 0

In [21]:
for row in range(4):
    line = ""
    for col in range(12):
        action = pi[row,col].argmax()
        action = {0: "👆", 1: "👇", 2: "👈", 3: "👉"}[action]
        line += action
    print(line)

👇👇👇👇👇👇👇👇👇👇👇👇
👇👇👇👇👇👇👇👇👇👇👇👇
👉👉👉👉👉👉👉👉👉👉👉👇
👆👆👆👆👆👆👆👆👆👆👆👆
